# Applied Data Science Capstone Week 4 The Battle of Neighborhoods

## A description of the problem and a discussion of the background

problem


## A description of the data and how it will be used to solve the problem.

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
# The code was removed by Watson Studio for sharing.

Import modules.

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import numpy as np # library to handle data in a vectorized manner

#aa
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import json # library to handle JSON files
import csv
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Download files

In [4]:
!wget -q -O 'List_of_postal_codes_of_Canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# postalcode - geographical coordinates
!wget -q -O 'geopostal.csv' https://cocl.us/Geospatial_data
    
!wget -q -O 'toronto.geojson' https://geo.nyu.edu/download/file/nyu-2451-36180-geojson.json
    
!wget -q -O 'newyork.geojson' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json 
!wget -q -O 'List_of_postal_codes_of_Canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# postalcode - geographical coordinates
!wget -q -O 'geopostal.csv' https://cocl.us/Geospatial_data    

## csv download

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###  Toronto

simple function to text in cell in table.

In [5]:
with open('toronto.geojson') as json_data:
    toronto_data = json.load(json_data)['features']
toronto_data

[{'geometry': {'coordinates': [-73.80597686767578, 47.87295150756836, 0],
   'type': 'Point'},
  'geometry_name': 'geom',
  'id': 'nyu_2451_36180.1',
  'properties': {'bbox': [-73.80597686767578,
    47.87295150756836,
    -73.80597686767578,
    47.87295150756836],
   'f_code': 'ZD045',
   'f_codedesc': 'Text Description',
   'id': 1,
   'symbol_id': 36,
   'text_angle': 0,
   'text_size': 0.00159,
   'textstring': 'FERRY / TRAVERSIER',
   'tile_id': 6,
   'txt_id': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-78.2855987548828, 46.5357551574707, 0],
   'type': 'Point'},
  'geometry_name': 'geom',
  'id': 'nyu_2451_36180.2',
  'properties': {'bbox': [-78.2855987548828,
    46.5357551574707,
    -78.2855987548828,
    46.5357551574707],
   'f_code': 'ZD045',
   'f_codedesc': 'Text Description',
   'id': 2,
   'symbol_id': 36,
   'text_angle': 0,
   'text_size': 0.00159,
   'textstring': 'Portage',
   'tile_id': 11,
   'txt_id': 1},
  'type': 'Feature'},
 {'geometry': {'coor

Load the downloaded wiki page to BeautifulSoup.

In [6]:
with open('List_of_postal_codes_of_Canada.html') as html_file:
    soup = BeautifulSoup(html_file,"lxml")

Declare a function and variables.

In [7]:
postal_to_geographica_coordinates = {}
with open('geopostal.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    
    for index, row in enumerate(csvreader):
        if index > 0 :  postal_to_geographica_coordinates.setdefault(row[0], (row[1],row[2]))
postal_to_geographica_coordinates   

{'M1B': ('43.8066863', '-79.1943534'),
 'M1C': ('43.7845351', '-79.1604971'),
 'M1E': ('43.7635726', '-79.1887115'),
 'M1G': ('43.7709921', '-79.2169174'),
 'M1H': ('43.773136', '-79.2394761'),
 'M1J': ('43.7447342', '-79.2394761'),
 'M1K': ('43.7279292', '-79.2620294'),
 'M1L': ('43.7111117', '-79.2845772'),
 'M1M': ('43.716316', '-79.2394761'),
 'M1N': ('43.692657', '-79.2648481'),
 'M1P': ('43.7574096', '-79.273304'),
 'M1R': ('43.7500715', '-79.2958491'),
 'M1S': ('43.7942003', '-79.2620294'),
 'M1T': ('43.7816375', '-79.3043021'),
 'M1V': ('43.8152522', '-79.2845772'),
 'M1W': ('43.7995252', '-79.3183887'),
 'M1X': ('43.8361247', '-79.2056361'),
 'M2H': ('43.8037622', '-79.3634517'),
 'M2J': ('43.7785175', '-79.3465557'),
 'M2K': ('43.7869473', '-79.385975'),
 'M2L': ('43.7574902', '-79.3747141'),
 'M2M': ('43.789053', '-79.4084928'),
 'M2N': ('43.7701199', '-79.4084928'),
 'M2P': ('43.7527583', '-79.4000493'),
 'M2R': ('43.7827364', '-79.4422593'),
 'M3A': ('43.7532586', '-79.329

Read text in table.

In [18]:
table_header=[ 'Neighborhood', 'PostalCode', 'Borough', 'Latitude', 'Longitude']
not_assigned = 'Not assigned'
table_values = []

def get_text_in_tag(tag):
    """
    get text in like <td>aaa</td> or <td><a>aaa</a></td> from argument tag.
    """
    if tag.string is not None:
        return tag.string
    else:
        return tag.find('a').string

In [19]:
postcode_dict = {} 
table_tag = soup.table
first_row = table_tag.find('tr')

for tr in first_row.find_next_siblings():
    row = [ get_text_in_tag(td) if td.string is None else td.string.strip() for td in tr.find_all('td')]

    if row[1] == not_assigned or row[1].find('Toronto') == -1:
        continue
    elif row[2] == not_assigned:
        postcode_dict.setdefault(row[1],  [row[0], row[1], postal_to_geographica_coordinates[row[0]] ])
    else:
        postcode_dict.setdefault(row[2],  [row[0], row[1], postal_to_geographica_coordinates[row[0]] ])
        
data_src = [ [key, value[0], value[1], float(value[2][0]), float(value[2][1]) ] for key,value in postcode_dict.items()]     

Make DataFrame.

In [20]:
toronto_data = pd.DataFrame(data_src, columns=table_header)
print(toronto_data.shape)
toronto_data.head()

(73, 5)


,Neighborhood,PostalCode,Borough,Latitude,Longitude
0,Ryerson,M5B,Downtown Toronto,43.657162,-79.378937
1,Runnymede,M6S,West Toronto,43.651571,-79.484450
2,Davisville,M4S,Central Toronto,43.704324,-79.388790
3,King and Spadina,M5V,Downtown Toronto,43.628947,-79.394420
4,Parkdale,M6R,West Toronto,43.648960,-79.456325


Get venues in Tronto via foursquare API.

In [21]:
toronto_venues = getNearbyVenues( names = toronto_data['Neighborhood'],
                                   latitudes = toronto_data['Latitude'],
                                   longitudes = toronto_data['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Ryerson
Runnymede
Davisville
King and Spadina
Parkdale
Dovercourt Village
Rosedale
Harbourfront West
Moore Park
Studio District
Brockton
Stn A PO Boxes 25 The Esplanade
Swansea
The Danforth West
South Niagara
The Annex
First Canadian Place
Berczy Park
India Bazaar
Davisville North
Harbourfront
Deer Park
Roselawn
Toronto Dominion Centre
Church and Wellesley
South Hill
King
Dufferin
Railway Lands
Central Bay Street
Yorkville
Forest Hill West
The Junction South
Forest Hill North
Summerhill West
Christie
Island airport
Regent Park
Grange Park
Bathurst Quay
CN Tower
Underground city
Exhibition Place
Summerhill East
Chinatown
Forest Hill SE
Business Reply Mail Processing Centre 969 Eastern
Garden District
North Toronto West
Union Station
Victoria Hotel
Toronto Islands
Riverdale
Rathnelly
Roncesvalles
St. James Town
Kensington Market
Adelaide
Cabbagetown
The Beaches West
Richmond
Design Exchange
Commerce Court
Parkdale Village
Lawrence Park
University of Toronto
Little Portugal
Harbord
North 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ryerson,43.657162,-79.378937,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
1,Ryerson,43.657162,-79.378937,Silver Snail Comics,43.657031,-79.381403,Comic Shop
2,Ryerson,43.657162,-79.378937,Burrito Boyz,43.656265,-79.378343,Burrito Place
3,Ryerson,43.657162,-79.378937,Blaze Pizza,43.656518,-79.380015,Pizza Place
4,Ryerson,43.657162,-79.378937,Tacos 101,43.656636,-79.376968,Taco Place


Find out how many unique categories can be curated from all the returned venues.

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


Analyze Each Neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(3241, 238)


,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ryerson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(73, 238)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.030000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000,0.000000,0.000000,0.000000,0

Get geolocator of Toronto.

In [28]:
address = 'Toronto city'

geolocator = Nominatim()
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.7394839, -79.369314.


Print each neighborhood along with the top 5 most common venues.

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3     Boat or Ferry  0.07
4          Boutique  0.07


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2    Restaurant  0.06
3        Bakery  0.04
4           Pub  0.04


----Brockton----
                    venue  freq
0             Coffee Shop  0.10
1          Breakfast Spot  0.10
2                    Café  0.10
3  Furniture / Home Store  0.05
4   Performing Arts Venue  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Smoke Shop  0.06
2                 Spa  0.06
3       Auto Workshop  0.06
4             Butcher  0.06


----CN Tower----
              ven

In [12]:
with open('newyork.geojson') as json_data:
    newyork_data = json.load(json_data)
    oad(json_data)['features']
newyork_data

AttributeError: module 'json' has no attribute 'l'

Tranform the data into a pandas dataframe

In [ ]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
def set_data_to_df(neighborhoods_data):
    neighborhoods = pd.DataFrame(columns=column_names)
    for data in neighborhoods_data:
        borough = neighborhood_name = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']

        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]

        neighborhoods = neighborhoods.append({'Borough': borough,
                                              'Neighborhood': neighborhood_name,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon}, ignore_index=True)

In [ ]:
# instantiate the dataframe
torontos_neighborhoods = set_data_to_df(toronto_data) 

In [ ]:
# instantiate the dataframe
newyork_neighborhoods = set_data_to_df(newyork_data) 

Load a downloaded wiki page to BeautifulSoup.

In [ ]:
with open('List_of_postal_codes_of_Canada.html') as html_file:
    soup = BeautifulSoup(html_file,"lxml")

Read the 2nd or later rows in table and create dictionary.
When postalcode as key is in dictionary alaready, add Neighbourhood to array in value of dictionary.

In [ ]:
postal_to_geographica_coordinates = {}
with open('geopostal.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    
    for index, row in enumerate(csvreader):
        if index > 0 :  postal_to_geographica_coordinates.setdefault(row[0], (row[1],row[2]))

In [ ]:
not_assigned = 'Not assigned'
table_header=[ 'Neighborhood', 'PostalCode', 'Borough', 'Latitude', 'Longitude', 'IsTront']

table_values = []
postcode_dict = {} 

table_tag = soup.table
first_row = table_tag.find('tr')

for tr in first_row.find_next_siblings():
    row = [ get_text_in_tag(td) if td.string is None else td.string.strip() for td in tr.find_all('td')]

    if row[1] == not_assigned:
        continue
    elif row[2] == not_assigned:
        postcode_dict.setdefault(row[1],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Tront') > -1  ])
    else:
        postcode_dict.setdefault(row[2],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Toronto') > -1  ])
        
data_src = [ [key, value[0], value[1], float(value[2][0]), float(value[2][1]), value[3] ] for key,value in postcode_dict.items()]  

Create a DataFrame. IsTront == True then the row is Toronto data,

In [ ]:
neighborhoods = pd.DataFrame(data_src, columns=table_header)
print(neighborhoods.shape)
neighborhoods.head()

Get latitude and longitude of Toronto and New York.

In [ ]:
geolocator = Nominatim()

In [ ]:
location = geolocator.geocode('Toronto city')
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

In [ ]:
location = geolocator.geocode('New York city')
newyork_latitude = location.latitude
newyork_longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(newyork_latitude, newyork_longitude))

In [ ]:
toronto_data = neighborhoods.loc[neighborhoods['IsTront'] == True]
toronto_data.head()

In [ ]:
newyork_data = neighborhoods.loc[neighborhoods['IsTront'] != True]
newyork_data

In [ ]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
print(len(toronto_data))     
map_toronto

In [ ]:
# create map of Manhattan using latitude and longitude values
map_newyork = folium.Map(location=[newyork_latitude, newyork_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(newyork_data['Latitude'], newyork_data['Longitude'], newyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
print(len(newyork_data))    
map_newyork

In [ ]:
# create map of Manhattan using latitude and longitude values
map_all = folium.Map(location=[newyork_latitude, newyork_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_all)  
print(len(neighborhoods))     
map_all